# 2. Q-learning

In the previous exercise we had complete knowledge of the dynamics of the environment. We had a model (through the `p` function) that predicts exactly what will happen in the future. For a lot of environments it is infeasible to get such a model. For these environments we can use an algorithm that is model-free. It interacts with the environment to discover which actions result in large returns and which don't. In this exercise we will implement the Q-learning algorithm using lookup tables.

First let's import the necessary components. We will again use the numpy library for working with vectors and matrices. The other library `gym` is a library for reinforcement learning environments. We will dive deeper into that first.

In [ ]:
import numpy as np
import gym

# Taxi

In this exercise we will use the [OpenAI Gym](https://gym.openai.com/) library. This library contains a lot of different reinforcement learning environments that you can use to evaluate your algorithms. The environments range from simple environments with only a few states, so complete video games ([Atari](https://github.com/openai/gym/blob/master/docs/environments.md#atari)) and 3D simulated robots ([MuJoCo](https://github.com/openai/gym/blob/master/docs/environments.md#mujoco)). All these environments use the same basic interface, which makes them very useful to compare and evaluate RL algorithms, making it the de-facto benchmark in the RL industry.

In this exercise we will use the `Taxi` environment and we will first explore how it works.

Let's create the environment first by simply calling `gym.make`.

In [ ]:
env = gym.make('Taxi-v3')
print(f'Observation: {env.observation_space}, {env.observation_space.dtype}')
print(f'Action:      {env.action_space}, {env.action_space.dtype}')

You can now see that the environment will return observations (a.k.a. state) with a shape `Discrete(500)`. This means that it is a single discrete value (i.e. integer) between 0 and 500. In other environments it can also return `Box()` spaces which will are multidimensional. The environment also accepts only a single action value between 0 and 6.

So this is a very simple environment that gives a single number and accepts a single number.

Furthermore, the environment supports rendering the current state so you can see what is going on. In this case it is a text-based representation of the world.

In [ ]:
env.render()

The world consists of a 5x5 grid with 4 pickup and dropoff positions (R,G,Y,B). There is a Taxi driving around in this world depicted with the filled square. It is yellow if there is no passenger on-board and green if there is. The passenger is waiting at the blue colored position and its destination position is colored pink. There are also some walls to make the route a bit more challenging.

The goal of the agent is to pickup the passenger and drop it off at its desired destination as fast as possible. The agent has 6 actions (in this order): moving south, north, east, west and pickup and dropoff of passenger.

The state the environment returns is a single number, which represent the entire state of the world. There are 5x5=25 possible states where the taxi can be, there are 5 states where the passenger can be (4 positions and on-board the taxi), and there are 4 destination states. So 5x5 x 5 x 4 = 500 different states.

You can interact with the environment in the following way.

First the environment must be reset, so it is initialized and at its start state. This can be achieved with the `env.reset()` function which will return the current state (observation). The start state is randomly chosen.

In [ ]:
observation = env.reset()
print(observation)
env.render()

Then the agent can perform an action by calling the `step(a)` function, which only required the action to be taken, which is in this case simply an integer value. It returns the next observation (state) after the action is performed, the resulting reward, a flag indicating if the episode is finished (done), and some extra information that can be discarded.

In [ ]:
observation, reward, done, _ = env.step(0)
env.render()
print(f'next observation: {observation}, reward: {reward}, done: {done}')

We don't know anything else about this environment. We don't know to which state an action will bring us, what the reward will be, or when an episode is finished. We only have this interface.

## Q-learning

<img src="figures/q-learning.png" width="50%" align="right"/> Since we don't know (or cannot query) what certain actions will do in certain states, we will have to discover that ourselves. For this we will use the Q-learning algorithm as described on the right.

This algorithm can be broken into a few smaller pieces. First of all we have to interact with environment in such a way that we nicely balance exploration and exploitation. If we would only follow our policy that acts greedy, than we have a big chance that it will perform sub-optimal. By keeping exploring we will figure out the optimal policy eventually.

We will use the $\epsilon$-greedy algorithm to combine exploring with our greedy policy. For every time step we will, with probability $\epsilon$, select an action completely at random. With probability $1-\epsilon$ we will select the best (i.e. greedy) action.

In order to determine which action is the best, we have to know the *action value function* $q_\pi(s,a)$. For this implementation we will store an estimate of that in a 2-dimensional array `q` for all the states and actions possible. This is feasible in this environment because we only have 500 states and 6 actions, resulting in a table of only 3000 elements. The rows in this array are the states and the columns the actions. The value for action `a` in state `s` can be queried with `q[s, a]` or if you need the values of all actions in state `s` you can grab the entire row with `q[s]`.

After every interaction with the environment we will update this table for action taken with the reward recieved. This way we will slowly (with factor $\alpha$) update the estimate towards the actual value.

### $\epsilon$-greedy policy
Now let's implement the $\epsilon$-greedy policy. We will need the function [`np.random.uniform()`](https://numpy.org/doc/stable/reference/random/generated/numpy.random.uniform.html) to get a random number between 0 and 1. Then we can sample a completely random action in the environment using `env.action_space.sample()`. You can use [`np.argmax()`](https://numpy.org/doc/stable/reference/generated/numpy.argmax.html) to get the index of the value with highest value in an array.

In [ ]:
def epsilon_greedy_policy(state, q, epsilon):
    ### START CODE ###
    ...
    ### END CODE ###
    return action

Let's see if it works as expected. If you run the next cell the output should be `3 1`. This means that in the first case it selects at random action `3`, and in the second case it selects the maximum action in state 1 (i.e. acted greedy), which is `1`.

In [ ]:
env.action_space.seed(12)
np.random.seed(12)

q = np.array([[1,1,3],[1,2,1]], dtype=np.float32)
a = epsilon_greedy_policy(1, q, 0.3)
b = epsilon_greedy_policy(1, q, 0.3)
print(a, b)

### Update Q values

Now we will implement the update of the table. After an interaction with the environment we get a new state and a reward. With this information we can create a new target value for the $q(s,a)$ estimate.

The next steps are based on a few assumptions. First of all, we assume that after this interaction we will follow only the greedy policy, so we can predict what will happen. Furthermore, we will assume that action values we have estimated so far are good estimates which we can use to make predictions, i.e. we assume they estimate the optimal function $q_*(s,a)$.

The Bellman optimality equation for the the action value function is described as follows.

$$
q_*(s,a) \doteq \mathop{\mathbb{E}} \left[ R_{t+1} + \gamma \underset{a'}\max {q_*(S_{t+1}, a')} \big| S_t=s,A_t=a \right]
$$

The reward $r$ and next state $s'$ are a sample of the environment, so if we slowly update our estimate for $q_\pi(s,a)$ using this sample, then we know we will slowly converge towards the optimal action value function. In other words, the target for our value function is

$$r + \gamma \underset{a'}\max {q_\pi(s',a')}$$

We want to slowly move towards that target, so we can implement this as follows:

$$q(s,a) = q(s,a) + \alpha [r + \gamma \underset{a'}\max {q_\pi(s',a')} - q(s,a)]$$

Time to implement this in `update_action_values` below. The current table is given as parameter `q` and we will replace the value for the current state-action pair with a new estimate. For this we will need the numpy function [`np.amax`](https://numpy.org/doc/stable/reference/generated/numpy.amax.html) to find the maximum value of an array. Remember that you can use `q[s]` to get the values of all actions in a state.

In [ ]:
def update_action_values(state, action, reward, next_state, q, alpha, gamma):
    ### START CODE ###
    q[state, action] = ...
    ### END CODE

Let's test the implemention with the following unit test.

In [ ]:
q = np.array([[1,1,3],[1,2,1]], dtype=np.float32)
print(q[0])
update_action_values(0, 2, -1, 1, q, 0.1, 0.9)
print(q[0])

The output should be equal to:

    [1. 1. 3.]
    [1.   1.   2.78]

In other words, the value for action 2 in state 0 should be updated as a result of reward -1.

## Learn

That's all the components we need for this algorithm. Time to put it all together.

We will implement a loop that runs for a number of episodes. For each interaction we use should use `epsilon_greedy_policy` and update the action value table using `update_action_values`.

Running this can take a few moments.

In [ ]:
alpha, gamma, epsilon = 0.1, 0.8, 0.3

q = np.zeros([env.observation_space.n, env.action_space.n])

for episode in range(5000):
    ### START CODE ###
    # Reset the environment
    state = ...
    done = False
    while not done:
        # Select an action using e-greedy policy
        action = ...
        
        # Perform a step in the environment
        next_state, reward, done, _ = ...

        # Update the action values table
        ...

        # Go to the next state
        state = next_state
        
    ### END CODE ###

Now we have learned a policy that should behave more or less optimal. Time to see it in action.

## Evaluate

To see how our policy performs we will run a complete episode with the greedy policy. We want to exploit completely, no more exploring.

So let's implement a greedy policy. This should simply find the action in the current state with the highest value. In other words, use the [`np.argmax`](https://numpy.org/doc/stable/reference/generated/numpy.argmax.html) function.

In [ ]:
def greedy_policy(state, q):
    ### START CODE ###
    action = ...
    ### END CODE ###
    return action

Now we can run an episode with this policy. Complete the following code and run it. This will show the total reward at the end, which should be positive.

In [ ]:
from IPython.display import clear_output
from time import sleep

total_reward = 0

state, done = env.reset(), False
while not done:
    clear_output(wait=True)
    env.render()
    sleep(0.1)

    ### START CODE ###
    # Select greedy action
    action = ...
    
    # Perform action in environment
    state, reward, done, _ = ...
    ### END CODE ###

    total_reward += reward
print(f'total reward: {total_reward}')

## Conclusion

That's it. You have now learned a policy that performs optimally without using a model to predict what happens in the environment. By simply interacting with the environment we learned which actions are good and which aren't.

The amount of computation needed is not that much, but that's mainly because we can use a table with limited number of entries. For environments with lots of states and/or actions this algorithm is no longer feasible. For that we need to use function approximations, which we will do in the next exercise.